In [1]:
import os
import re
import sys
import time
import datetime
import time
import json
import html
import torch
import gensim
import random
import transformers
import pandas as pd
import numpy as np
import urllib.request
import torch.nn as nn
from kiwipiepy import Kiwi
from bs4 import BeautifulSoup
from selenium import webdriver
import torch.nn.functional as F
from transformers import pipeline
from gensim.models import Word2Vec
from kiwipiepy.utils import Stopwords
from selenium.webdriver.common.by import By
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
#seed 고정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [366]:
# CSV 파일 불러오기  
filename = 'news_articles_후쿠시마.csv'
mlp_keyword = pd.read_csv(filename)

# 파일명에서 키워드 추출 (예: 후쿠시마 오염수,윤석열,이재명....)
keyword = os.path.splitext(filename)[0].split('_')[-1]

# 진보,보수 성향 및 기타 언론사 리스트
progressive_media = ['한겨레', '경향신문', '오마이뉴스']
conservative_media = ['조선일보','동아일보','중앙일보']
test_media = ['연합뉴스', '매일경제', '머니투데이']

# 진보 성향 데이터프레임
progressive_df = mlp_keyword[mlp_keyword['Source'].isin(progressive_media)]

# 보수 성향 데이터프레임
conservative_df = mlp_keyword[mlp_keyword['Source'].isin(conservative_media)]

# test 언론사 데이터프레임
test_media_df = mlp_keyword[mlp_keyword['Source'].isin(test_media)]


# Progressive = 1, Conservative = 0
progressive_df['Label'] = 1
conservative_df['Label'] = 0

other_media = ['SBS Biz','한국경제TV','뉴스1','JTBC','채널A','MBN','SBS','MBC','YTN','뉴시스', 
                'KBS','TV조선','연합뉴스TV','매경이코노미','한경비즈니스','월간산','한겨레21',
                '이코노미스트','더스쿠프','주간경향','신동아','시사저널','주간조선','시사IN',
                '주간동아','농민신문','기자협회보','코메디닷컴','코리아헤럴드','코리아중앙데일리',
                '일다','뉴스타파','동아사이언스','헬스조선','여성신문','서울신문','문화일보','세계일보',
                '한국일보','국민일보','아이뉴스24','디지털데일리','데일리안','머니S','블로터','더팩트',
                '프레시안','지디넷코리아','미디어오늘','디지털타임스','노컷뉴스','전자신문','서울경제',
                '헤럴드경제','비즈워치','아시아경제','파이낸셜뉴스','조선비즈','이데일리','조세일보','한국경제']

# 전체 언론사명 리스트
all_media = progressive_media + conservative_media + test_media + other_media

C:\Users\7johs\AppData\Local\Temp\ipykernel_22732\4221925947.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  progressive_df['Label'] = 1
C:\Users\7johs\AppData\Local\Temp\ipykernel_22732\4221925947.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conservative_df['Label'] = 0


In [367]:
# 기사 제목에서 []로 감싸져 있는 텍스트 및 언론사 이름 제거 함수
def remove_bracketed_text_and_media(title, media_list):
    title = re.sub(r'\[.*?\]', '', title).strip()
    for media in media_list:
        title = title.replace(media, '').strip()
    return title

# 모든 데이터프레임에서 제목 수정
progressive_df['Title'] = progressive_df['Title'].apply(remove_bracketed_text_and_media, media_list=all_media)
conservative_df['Title'] = conservative_df['Title'].apply(remove_bracketed_text_and_media, media_list=all_media)
test_media_df['Title'] = test_media_df['Title'].apply(remove_bracketed_text_and_media, media_list=all_media)

C:\Users\7johs\AppData\Local\Temp\ipykernel_22732\716994766.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  progressive_df['Title'] = progressive_df['Title'].apply(remove_bracketed_text_and_media, media_list=all_media)
C:\Users\7johs\AppData\Local\Temp\ipykernel_22732\716994766.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conservative_df['Title'] = conservative_df['Title'].apply(remove_bracketed_text_and_media, media_list=all_media)
C:\Users\7johs\AppData\Local\Temp\ipykernel_22732\716994766.py:

In [368]:
# 한국어 BERT 모델을 사용하여 입력된 문장을 변형 (Data Augmentation)
# random_masking_replacement : 무작위로 단어를 마스킹하고 복원하는 방식
# random_masking_insertion : 문장 내에 무작위로 마스크 토큰을 삽입하고 이를 복원하는 방식


class BERT_Augmentation:
    def __init__(self):
        self.model_name = 'monologg/koelectra-base-v3-generator'
        self.model = transformers.AutoModelForMaskedLM.from_pretrained(self.model_name)
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(self.model_name)
        self.unmasker = pipeline("fill-mask", model=self.model, tokenizer=self.tokenizer)
        random.seed(42)

    # 무작위로 단어를 마스킹하고 복원하는 방식
    def random_masking_replacement(self, sentence, ratio=0.15):
        span = int(round(len(sentence.split()) * ratio))
        
        # 품질 유지를 위해, 문장의 어절 수가 4 이하라면 원문장을 그대로 리턴
        if len(sentence.split()) <= 4:
            return sentence

        mask = self.tokenizer.mask_token
        unmasker = self.unmasker

        unmask_sentence = sentence
        # 처음과 끝 부분을 [MASK]로 치환 후 추론할 때의 품질이 좋지 않음
        # 따라서 처음 단어(0번째)와 끝 단어를 제외하고 무작위 인덱스를 선택함
        random_idx = random.randint(1, len(unmask_sentence.split()) - span)
        
        unmask_sentence = unmask_sentence.split()
        cache = []
        for _ in range(span):
            # 처음과 끝 부분을 [MASK]로 치환 후 추론할 때의 품질이 좋지 않음
            # 따라서 처음 단어(0번째)와 끝 단어를 제외하고 무작위 인덱스를 선택함
            while cache and random_idx in cache:
                random_idx = random.randint(1, len(unmask_sentence) - 2)
            cache.append(random_idx)
            unmask_sentence[random_idx] = mask
            unmask_sentence = unmasker(" ".join(unmask_sentence))[0]['sequence']
            unmask_sentence = unmask_sentence.split()
        unmask_sentence = " ".join(unmask_sentence)
        unmask_sentence = unmask_sentence.replace("  ", " ")

        return unmask_sentence.strip()
    
    # 문장 내에 무작위로 마스크 토큰을 삽입하고 이를 복원하는 방식
    def random_masking_insertion(self, sentence, ratio=0.15):
        span = int(round(len(sentence.split()) * ratio))
        mask = self.tokenizer.mask_token
        unmasker = self.unmasker
        
        # Recover
        unmask_sentence = sentence
        
        for _ in range(span):
            unmask_sentence = unmask_sentence.split()
            random_idx = random.randint(0, len(unmask_sentence)-1)
            unmask_sentence.insert(random_idx, mask)
            unmask_sentence = unmasker(" ".join(unmask_sentence))[0]['sequence']

        unmask_sentence = unmask_sentence.replace("  ", " ")

        return unmask_sentence.strip()

In [369]:
# BERT 증강 수행
augmenter = BERT_Augmentation()

# 진보(progressive)와 보수(conservative) 데이터의 수를 비교
# 데이터 수가 적은 쪽의 수를 증가시키기 위해 증강할 데이터 수 결정

# 데이터 프레임에서 무작위로 문장을 선택하여, 두 가지 증강 방법 사용
# random_masking_replacement: 문장에서 무작위로 단어를 마스킹하고 복원하는 방식
# random_masking_insertion: 문장 내에 무작위로 마스크 토큰을 삽입하고 이를 복원하는 방식

# 선택된 각 문장에 대해 두 가지 증강 방법을 각각 한 번씩 적용하여 새로운 두 개의 문장을 생성
# 만약 증강해야 될 데이터 수가 홀수이면 첫 번째 증강 방법(random_masking_replacement)만 한 번 더 수행
# 이렇게 생성된 문장들을 데이터프레임에 추가

# 최종적으로 데이터 수가 같아짐
def augment_dataframe(df, augmenter, augment_count):
    new_rows = []
    for _ in range(augment_count // 2):  # Each iteration adds two new rows
        random_idx = random.randint(0, len(df) - 1)
        original_text = df.iloc[random_idx]['Title']
        
        augmented_text = augmenter.random_masking_replacement(original_text)
        new_row = df.iloc[random_idx].copy()
        new_row['Title'] = augmented_text
        new_rows.append(new_row)
        
        augmented_text = augmenter.random_masking_insertion(original_text)
        new_row = df.iloc[random_idx].copy()
        new_row['Title'] = augmented_text
        new_rows.append(new_row)

    if augment_count % 2 != 0:
        random_idx = random.randint(0, len(df) - 1)
        original_text = df.iloc[random_idx]['Title']
        augmented_text = augmenter.random_masking_replacement(original_text)
        new_row = df.iloc[random_idx].copy()
        new_row['Title'] = augmented_text
        new_rows.append(new_row)
        
    return pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)


# 진보와 보수 데이터 크기 비교 및 증강할 데이터 수 결정
progressive_count = len(progressive_df)
conservative_count = len(conservative_df)

if progressive_count < conservative_count:
    augment_count = conservative_count - progressive_count
    progressive_df = augment_dataframe(progressive_df, augmenter, augment_count)
else:
    augment_count = progressive_count - conservative_count
    conservative_df = augment_dataframe(conservative_df, augmenter, augment_count)

# 데이터프레임 결합
combined_df = pd.concat([progressive_df, conservative_df], ignore_index=True)

# 증강된 문장 출력
print("증강된 문장 예시:")
for i in range(5):
    original_text = progressive_df.iloc[i]['Title']
    augmented_text_replacement = augmenter.random_masking_replacement(original_text)
    augmented_text_insertion = augmenter.random_masking_insertion(original_text)
    print(f"Original: {original_text}")
    print(f"Augmented (Replacement): {augmented_text_replacement}")
    print(f"Augmented (Insertion): {augmented_text_insertion}")
    print()

# 결과 확인
print("진보 성향 데이터 수:", len(progressive_df))
print("보수 성향 데이터 수:", len(conservative_df))
print("통합 데이터프레임 데이터 수:", len(combined_df))


증강된 문장 예시:
Original: 기시다 만나러 서울 가는 이용수 할머니 "법적 배상 촉구할 것"
Augmented (Replacement): 기시다 만나러 서울 가는 이용수 할머니 " 배상할 것 "
Augmented (Insertion): 기시다 만나러 서울 가는 이용수 할머니 " 법적인 배상을 촉구할 것 "

Original: 일본 오염수 방류 규탄 결의문 읽다가... 강제 퇴장당한 시의원
Augmented (Replacement): 일본 오염수 방류 규탄 결의문 읽다가.... 퇴장당한 시의원
Augmented (Insertion): 일본 오염수 방류 규탄 결의문 읽다가.... 강제 퇴장당한 시의원

Original: "'방사성 오염수 투기' 일본을 국제해양법 재판소에 제소하라"
Augmented (Replacement): "'방사성 오염수 투기'일본을 국제해양법 재판소에 "
Augmented (Insertion): "'방사성 오염수 투기'일본을 국제 국제해양법 재판소에 제소하라 "

Original: 한·중·러만 '오염수'로 부른다?... 미국이 오히려 예외
Augmented (Replacement): 한 · 중 · 러만'오염수'로 부른다?.... 오히려 예외
Augmented (Insertion): 한 · 중 · 러만'오염수'로만 부른다?... 미국이 오히려 예외

Original: 이종호 <> 기자, '올해의 주목할만한 데이터저널리스트상' 수상
Augmented (Replacement): 이종호 < > 기자,'올해의 주목할만한 데이터저널리스트상'발표
Augmented (Insertion): 이종호 < > 기자, 올해'올해의 주목할만한 데이터저널리스트상'수상

진보 성향 데이터 수: 848
보수 성향 데이터 수: 848
통합 데이터프레임 데이터 수: 1696


In [370]:
# Kiwi 초기화 및 불용어 설정
kiwi = Kiwi(typos='basic')
stopwords = Stopwords()

In [371]:
# 텍스트 전처리 함수
def clean_text(text):
    text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', '', text) #한글과 공백을 제외한 모든 문자 제거
    text = re.sub(r'\s+', ' ', text) #연속된 공백을 단일 공백으로 변환
    return text.strip() #양쪽 끝의 공백 문자 제거


# 토큰화 및 불용어 제거 함수
def tokenize_and_remove_stopwords(text, kiwi, stopwords):
    tokens = kiwi.tokenize(text, stopwords=stopwords)
    return [token.form for token in tokens]

In [372]:
# 전처리 및 토큰화
def preprocess_dataframe(df):
    df['Clean_Title'] = df['Title'].apply(clean_text)
    df['Tokenized_Title'] = df['Clean_Title'].apply(lambda x: ' '.join(tokenize_and_remove_stopwords(x, kiwi, stopwords)))
    return df

combined_df = preprocess_dataframe(combined_df)
test_media_df = preprocess_dataframe(test_media_df)

# 'Clean_Title' 값이 없는 행 제거
combined_df = combined_df[combined_df['Clean_Title'].str.strip() != '']
test_media_df = test_media_df[test_media_df['Clean_Title'].str.strip() != '']

C:\Users\7johs\AppData\Local\Temp\ipykernel_22732\1831875621.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Clean_Title'] = df['Title'].apply(clean_text)
C:\Users\7johs\AppData\Local\Temp\ipykernel_22732\1831875621.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tokenized_Title'] = df['Clean_Title'].apply(lambda x: ' '.join(tokenize_and_remove_stopwords(x, kiwi, stopwords)))


In [373]:
# 전처리 된 언론사별 데이터 수 계산 함수
def count_articles_by_media(df, media_list):
    return df[df['Source'].isin(media_list)]['Source'].value_counts()

# 각 미디어 그룹의 데이터프레임에서 언론사별 데이터 수 계산
combined_df_counts = count_articles_by_media(combined_df, progressive_media+conservative_media)
test_media_df_counts = count_articles_by_media(test_media_df, test_media)

display(combined_df_counts,test_media_df_counts)

Source
동아일보     524
한겨레      336
경향신문     290
중앙일보     244
오마이뉴스    222
조선일보      80
Name: count, dtype: int64

Source
매일경제     179
머니투데이    143
연합뉴스       1
Name: count, dtype: int64

In [374]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(combined_df['Tokenized_Title'])

X = tfidf_matrix
y = combined_df['Label']

In [ ]:
''' 
# Word2Vec 적용 시 코드

# Word2Vec 모델 학습
sentences = combined_df['Tokenized_Title'].tolist()
word2vec_model = gensim.models.Word2Vec(sentences, vector_size=100, window=2, min_count=1, workers=4)

# 문서 벡터화 함수
def get_document_vector(tokens, model):
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)
    
# 문서 벡터화
combined_df['Vector'] = combined_df['Tokenized_Title'].apply(lambda x: get_document_vector(x, word2vec_model))
test_media_df['Vector'] = test_media_df['Tokenized_Title'].apply(lambda x: get_document_vector(x, word2vec_model))

X = np.vstack(combined_df['Vector'].values)
y = combined_df['Label'].values
'''

In [375]:
# 학습 및 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 텐서로 변환
X_train_tensor = torch.tensor(X_train.toarray(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test.toarray(), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# DataLoader 생성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
''' 
#Word2Vec 적용 시 코드

# 학습 및 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 텐서로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# DataLoader 생성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
'''

In [376]:
# 모델 정의
class ImprovedNet(nn.Module):
    def __init__(self, in_dim, h1_dim, h2_dim, h3_dim, h4_dim, out_dim):
        super(ImprovedNet, self).__init__()
        self.fc1 = nn.Linear(in_dim, h1_dim)
        self.bn1 = nn.BatchNorm1d(h1_dim)
        self.fc2 = nn.Linear(h1_dim, h2_dim)
        self.bn2 = nn.BatchNorm1d(h2_dim)
        self.fc3 = nn.Linear(h2_dim, h3_dim)
        self.bn3 = nn.BatchNorm1d(h3_dim)
        self.fc4 = nn.Linear(h3_dim, h4_dim)
        self.bn4 = nn.BatchNorm1d(h4_dim)
        self.fc5 = nn.Linear(h4_dim, out_dim)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.bn2(self.fc2(x)))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.bn3(self.fc3(x)))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.bn4(self.fc4(x)))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc5(x)
        return x

In [377]:
# GPU 사용 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ImprovedNet(in_dim=X_train.shape[1], h1_dim=512, h2_dim=256, h3_dim=128, h4_dim=64, out_dim=2).to(device)

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

# 모델 학습
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/30], Loss: 0.7315
Epoch [2/30], Loss: 0.6741
Epoch [3/30], Loss: 0.5730
Epoch [4/30], Loss: 0.4171
Epoch [5/30], Loss: 0.3184
Epoch [6/30], Loss: 0.2177
Epoch [7/30], Loss: 0.1592
Epoch [8/30], Loss: 0.1272
Epoch [9/30], Loss: 0.1008
Epoch [10/30], Loss: 0.0750
Epoch [11/30], Loss: 0.0849
Epoch [12/30], Loss: 0.0550
Epoch [13/30], Loss: 0.0531
Epoch [14/30], Loss: 0.0358
Epoch [15/30], Loss: 0.0405
Epoch [16/30], Loss: 0.0346
Epoch [17/30], Loss: 0.0381
Epoch [18/30], Loss: 0.0396
Epoch [19/30], Loss: 0.0331
Epoch [20/30], Loss: 0.0346
Epoch [21/30], Loss: 0.0388
Epoch [22/30], Loss: 0.0333
Epoch [23/30], Loss: 0.0281
Epoch [24/30], Loss: 0.0397
Epoch [25/30], Loss: 0.0198
Epoch [26/30], Loss: 0.0175
Epoch [27/30], Loss: 0.0363
Epoch [28/30], Loss: 0.0296
Epoch [29/30], Loss: 0.0310
Epoch [30/30], Loss: 0.0386


In [378]:
# 모델 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 85.59%


In [ ]:
# test_media_df로 softmax 값 도출
test_media_tfidf_matrix = vectorizer.transform(test_media_df['Tokenized_Title'])

test_media_X_tensor = torch.tensor(test_media_tfidf_matrix.toarray(), dtype=torch.float32)
test_media_dataset = TensorDataset(test_media_X_tensor)
test_media_loader = DataLoader(test_media_dataset, batch_size=32, shuffle=False)

In [ ]:
'''
# Word2Vec 적용 시 코드

# test_media_df 벡터화
test_media_X = np.vstack(test_media_df['Vector'].values)

# 텐서로 변환
test_media_X_tensor = torch.tensor(test_media_X, dtype=torch.float32)
test_media_dataset = TensorDataset(test_media_X_tensor)
test_media_loader = DataLoader(test_media_dataset, batch_size=32, shuffle=False)
'''

In [379]:
# 예측 수행 및 softmax 값 도출
model.eval()
softmax = nn.Softmax(dim=1)
predictions = []
with torch.no_grad():
    for inputs in test_media_loader:
        inputs = inputs[0].to(device)
        outputs = model(inputs)
        probs = softmax(outputs)
        predictions.extend(probs.cpu().numpy())

# 결과 데이터프레임에 추가
test_media_df['Progressive_Probability'] = [p[1] for p in predictions]
test_media_df['Conservative_Probability'] = [p[0] for p in predictions]


# test meida 언론사별 평균 softmax 값과 데이터 수 계산
result = test_media_df.groupby('Source').agg(
    Progressive_Probability=('Progressive_Probability', 'mean'),
    Conservative_Probability=('Conservative_Probability', 'mean'),
    Count=('Source', 'size')
).reset_index()

result

,Source,Progressive_Probability,Conservative_Probability,Count
0,매일경제,0.755188,0.244812,179
1,머니투데이,0.733527,0.266473,143
2,연합뉴스,0.823134,0.176866,1


In [380]:
# 결과 파일 저장
result_filename = f"test_media_predictions(TF-IDF)_{keyword}_accuracy_{test_accuracy:.2f}.csv"
result.to_csv(result_filename, index=False)
print(f"Result saved to {result_filename}")

Result saved to test_media_predictions(TF-IDF)_후쿠시마 오염수_accuracy_85.59.csv


In [ ]:
'''
# Word2Vec 적용 시 코드

# 결과 파일 저장
result_filename = f"test_media_predictions(Word2Vec)_{keyword}_accuracy_{test_accuracy:.2f}.csv"
result.to_csv(result_filename, index=False)
print(f"Result saved to {result_filename}")
'''